In [2]:
import re
import json
import text_processing
import secrets 

from typing import List, Dict
import openai  # Assuming OpenAI API, but adaptable to other models

# Load text file and taxonomy JSON

taxonomy_file_path = 'privacy_ontology_simple.json'  # Replace with actual path
example_file_path = 'annotations/Actual_Budget/Accounts_&_Transactions.txt'
target_file_path = 'input/lh-ehr/Direct_Messaging_README.txt'

# Load previously processed example files
example_data = text_processing.process_input(example_file_path)

### Building prompts


In [3]:

import json
from typing import Dict, List, Union
import tiktoken  # Library for token counting with OpenAI models
import prompt_templates

# Load the privacy ontology
ontology_path = "privacy_ontology_simple.json"
privacy_ontology = prompt_templates.load_privacy_ontology(ontology_path)

# Load one processed example file for demonstration
example_file = text_processing.process_input(example_file_path)[0]


# Define target text to be annotated (for demonstration) & Load annotated version 
target_file_annotations = text_processing.process_input(target_file_path)
new_text_to_annotate = open(target_file_path).read()

# Generate the prompt
prompt_example = prompt_templates.create_annotation_prompt(example_file, new_text_to_annotate, privacy_ontology)

# Print the prompt to review it
print(prompt_example)
print(f"\nToken Count: {prompt_templates.count_tokens(prompt_example)} tokens")


You are a privacy expert annotator tasked with annotating text files with metadata about privacy behaviors and stories. For the given text, annotate the following:

1. Actions: Actions performed or expected in the text.
2. Data Types: Types of data referenced in the text. Data types may include specific subcategories.
3. Purposes: Intentions or purposes related to the actions and data types.
4. Stories: Concise stories that describe how actions, data types, and purposes interact in context.

After providing your annotations, explain your rationale for these annotations. Place <R> tag between your annotations and your rationale.

Use only the categories listed below when annotating:

Actions:
Collect, Use, Share

Data Types:
Contact Data:
  Phone Number:
  Email address:
  User ID:
  Job Title:
  Company:
  Address:
  Name:
  Date of Birth:
  Image:
  Government ID:
  Biographical Data:
    CV:
    Education:
    Employment:
Health Data:
  Physical activity:
Social Media:
Location:
  Ap

### Annotating with LLMs 

In [1]:
import os
import json
import prompt_templates
import text_processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

input_dir = 'input'

def find_matching_file(input_file_path, annotations_dir):
    """
    Find the matching annotation file for a given input file.
    
    Args:
        input_file_path (str): Path to the input file
        annotations_dir (str): Root directory of annotations
    
    Returns:
        str: Path to the matching annotation file, or None if not found
    """
    # Get the relative path from the input directory
    relative_path = os.path.relpath(input_file_path, input_dir)
    annotation_file_path = os.path.join(annotations_dir, relative_path)
    
    return annotation_file_path if os.path.exists(annotation_file_path) else None


def find_most_similar_file(input_file_path, annotations_dir, exclude_file):
    """
    Find the most similar annotation file to the input file, excluding a specified file.
    
    Args:
        input_file_path (str): Path to the input file
        annotations_dir (str): Directory containing the annotation files
        exclude_file (str): File to exclude from similarity search
    
    Returns:
        str: Path to the most similar annotation file
    """
    # Read the input file's content
    with open(input_file_path, 'r', encoding='utf-8') as f:
        input_text = f.read()
    
    # List to store (similarity_score, annotation_file_path)
    similarity_scores = []
    
    # Loop through the annotations directory to calculate similarity
    for root, _, files in os.walk(annotations_dir):
        for filename in files:
            if filename == exclude_file or filename.startswith('.'):
                continue
            
            annotation_file_path = os.path.join(root, filename)
            
            # Read the annotation file's content
            with open(annotation_file_path, 'r', encoding='utf-8') as f:
                annotation_text = f.read()
            
            # Compute similarity using TF-IDF and cosine similarity
            tfidf = TfidfVectorizer().fit_transform([input_text, annotation_text])
            similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
            similarity_scores.append((similarity, annotation_file_path))
    
    # Sort by similarity score in descending order and return the most similar file
    similarity_scores.sort(reverse=True, key=lambda x: x[0])
    return similarity_scores[0][1] if similarity_scores else None


def create_prompt_templates_dict(input_dir='input', annotations_dir='annotations', ontology_path='privacy_ontology_simple.json'):
    """
    Create a dictionary of prompt templates matching input files with their corresponding annotation examples.
    
    Args:
        input_dir (str): Root directory containing input files to be annotated
        annotations_dir (str): Root directory containing annotated example files
        ontology_path (str): Path to the privacy ontology JSON file
    
    Returns:
        dict: Dictionary of prompt templates for each input file
    """
    # Load privacy ontology
    privacy_ontology = prompt_templates.load_privacy_ontology(ontology_path)
    
    # Dictionary to store prompt templates
    prompt_templates_dict = {}
    
    # Walk through all directories and files in the input directory
    for root, _, files in os.walk(input_dir):
        for filename in files:
            # Skip hidden files
            if filename.startswith('.'):
                continue
            
            # Full path to the input file
            input_file_path = os.path.join(root, filename)
            
            # Find corresponding annotation file (for exclusion from similarity search)
            annotation_file_path = find_matching_file(input_file_path, annotations_dir)
            
            if not annotation_file_path:
                print(f"No matching annotation found for {input_file_path}")
                continue
            
            try:
                # Read the input text to annotate
                with open(input_file_path, 'r', encoding='utf-8') as f:
                    new_text_to_annotate = f.read()
                
                # Find the most similar annotation file
                most_similar_annotation = find_most_similar_file(input_file_path, annotations_dir, os.path.basename(annotation_file_path))
                
                if not most_similar_annotation:
                    print(f"No similar annotation found for {input_file_path}")
                    continue
                
                # Process the example file from annotations
                example_file = text_processing.process_input(most_similar_annotation)[0]
                
                # Create prompt template
                prompt_template = prompt_templates.create_0_shot_annotation_prompt(
                    example_file, 
                    new_text_to_annotate, 
                    privacy_ontology
                )
                
                # Create a unique key based on relative path
                relative_key = os.path.relpath(input_file_path, input_dir)
                
                # Store in dictionary
                prompt_templates_dict[relative_key] = {
                    'input_file_path': input_file_path,
                    'annotation_file_path': most_similar_annotation,
                    'prompt_template': prompt_template,
                    'target_annotations': text_processing.process_file(annotation_file_path),
                    'token_count': prompt_templates.count_tokens(prompt_template)
                }
            
            except Exception as e:
                print(f"Error processing {input_file_path}: {e}")
    
    return prompt_templates_dict


# Create the prompt templates dictionary
prompt_templates_dict = create_prompt_templates_dict()

# Save to JSON for inspection
with open('prompt_templates_output.json', 'w', encoding='utf-8') as f:
    json.dump({k: {**v, 'prompt_template': v['prompt_template']} 
              for k, v in prompt_templates_dict.items()}, 
              f, indent=2)

print(f"Created prompt templates for {len(prompt_templates_dict)} files")

# Optional: You can save the full dictionary using pickle if needed
import pickle
with open('prompt_templates.pkl', 'wb') as f:
    pickle.dump(prompt_templates_dict, f)


Error processing input\Actual_Budget\Backup_&_Restore.txt: 'charmap' codec can't decode byte 0x9d in position 232: character maps to <undefined>
Created prompt templates for 24 files


In [4]:
import os
import csv
import json
from getpass import getpass
import model_routing
from typing import Dict, List

# Ensure OPENAI_API_KEY is set
# os.environ['OPENAI_API_KEY'] = getpass('Enter your OPENAI API key: ')

os.environ['GROQ_API_KEY'] = getpass('Enter your Groq API key: ')

def save_results_to_csv(models: List[str], 
                         prompt_templates_dict: Dict, 
                         model_responses: Dict, 
                         output_file: str):
    """
    Save the model outputs, prompts, and file annotations to a CSV file.
    """
    # Open the output CSV file
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        # Prepare the CSV header
        fieldnames = ['File', 'Prompt', 'Model', 'Target File Path', 'Target Annotations', 'Model Response 1', 'Model Response 2']
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Create a list of file keys to maintain order
        file_keys = list(prompt_templates_dict.keys())

        # Iterate through the prompt templates and models to pair responses
        for file_key in file_keys:
            template_info = prompt_templates_dict[file_key]
            
            for model in models:
                # Get responses for this model
                model_specific_responses = model_responses.get(model, {})
                
                # Get responses for this specific file/prompt
                prompt_responses = model_specific_responses.get(file_key, [])
                
                # Ensure we have at least two responses (or 'No Response')
                response1 = prompt_responses[0] if prompt_responses and len(prompt_responses) > 0 else 'No Response'
                response2 = prompt_responses[1] if prompt_responses and len(prompt_responses) > 1 else 'No Response'

                # Create a row for each model and file
                row = {
                    'File': file_key,
                    'Prompt': template_info['prompt_template'],
                    'Model': model,
                    'Target File Path': template_info['input_file_path'],
                    'Target Annotations': json.dumps(template_info['target_annotations']),
                    'Model Response 1': json.dumps(response1, ensure_ascii=False),
                    'Model Response 2': json.dumps(response2, ensure_ascii=False)
                }

                # Write the row to the CSV file
                writer.writerow(row)


def run_multi_file_annotations(prompt_templates_dict, output_csv, models=None):
    """
    Run annotation process for multiple files and save results to a CSV file.
    """

    try:
        # Create a list of file keys to maintain order
        file_keys = list(prompt_templates_dict.keys())

        # Prepare prompts that maintains the file key order
        test_prompts = [
            prompt_templates_dict[file_key]['prompt_template'] 
            for file_key in file_keys
        ]

        # Get model responses 
        raw_model_responses = model_routing.run_multi_model_prompts(
            models=models, 
            prompts=test_prompts, 
            num_runs=2  # Number of runs per model
        )

        # Restructure responses to match file keys
        model_responses = {}
        for model in models:
            model_responses[model] = {
                file_key: model_runs 
                for file_key, model_runs in zip(file_keys, raw_model_responses.get(model, []))
            }

        # Save results to CSV 
        save_results_to_csv(
            models=models,
            prompt_templates_dict=prompt_templates_dict,
            model_responses=model_responses,
            output_file=output_csv
        )

        print(f"Annotation results saved to {output_csv}")
        print(f"Number of files processed: {len(prompt_templates_dict)}")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

# Example usage remains the same
output_csv = 'LLMAnnotation_grokLLama3_2_0_shot.csv'
models = [
    # 'openai:gpt-4o-2024-11-20'
    'groq:llama-3.2-3b-preview'
    ]
run_multi_file_annotations(prompt_templates_dict, output_csv, models=models)

Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-preview - Completed a run
Model: groq:llama-3.2-3b-previe

In [4]:


import aisuite as ai
import os 
# Initialize the AI client for accessing the language model
client = ai.Client()
os.environ["GROQ_API_KEY"] = "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"

# Define a conversation with a system message and a user message
messages = [
    {"role": "system", "content": "You are a helpful agent, who answers with brevity. Please rewrite the following paper review to be in actionable human language with complete sentences using similar and identical language to the input where possible"},
    {"role": "user", "content": "Privacy policies have obligation to be clear i.e. regulations, this paper proposes a method of scoring privacy policies to determine their risks contextual integrity - control over information and guarantee of proper data flows lattice - based: identify minimum PI collection and usage that enables application functionality to build the lattice: 1. privacy practice identification What is the most sensitive information collected by this social network, by default? • How long is that most sensitive information retained? • With whom is that most sensitive information shared? • By default, to what level of detail can authorized individuals view the most sensitive information? 2. CI analysis identify privacy statements (descriptions of the data flow of the application) i.e., subject (user), senders, attributes (data types), recipients, Transimission principles (desc of inf flow) -> Evaluate on contextual completeness (if a statements contains all 5 contextual parameters) to generate a CI score 3. Lattice analysis Privacy practices are used to build edges in the lattice representation of these policies where they are built into retention, visibility, and Granularity lattices where the data types retention, visibility (who has access) and granularity (from the privacy practices) are marked. Given the paths and their weights generated in these lattices through the identified privacy practices a score is given where a higher-score indicates better privacy preserving practices. lScore and CiScore were used to evaluate several osn's for their practices and privacy policy clarity. These scores were normalized to rank each policy by their potential privacy violations and communication of privacy practices."},
]

# Request a response from the model
response = client.chat.completions.create(model="groq:llama-3.3-70b-versatile", messages=messages)

# Print the model's response
print(response.choices[0].message.content)

This paper proposes a method to evaluate the effectiveness of privacy policies by scoring them based on their risks and clarity. To do this, the method uses a lattice-based approach to identify the minimum amount of personal information that needs to be collected and used for an application to function. 

The evaluation process involves two main steps: 
1. Identifying privacy practices by answering questions such as: What is the most sensitive information collected by this social network by default? How long is this information retained? With whom is it shared? And to what level of detail can authorized individuals view this information? 
2. Analyzing these practices to generate a Contextual Integrity (CI) score and a Lattice score. 

The CI score is calculated by evaluating the completeness of privacy statements, which include descriptions of the data flow, such as the subject, sender, attributes, recipients, and transmission principles. A higher CI score indicates that the statement 